In [20]:
import pandas as pd
import numpy as np

In [10]:
from pyspark.sql import SparkSession
#Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Liveability")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)


In [30]:
df = spark.read.parquet('../data/landing0/combined_data_with_geolocation.parquet', header=True, inferSchema=True)
df1 = spark.read.parquet('../data/landing/affordability.parquet', header=True, inferSchema=True)
df1 = df1.withColumnRenamed("postcode", "Postcode")
df1.show()

+--------+--------------------------------+-------------------------------+
|Postcode|1-2 Bedder Affordability Metrics|3+ Bedder Affordability Metrics|
+--------+--------------------------------+-------------------------------+
|    3206|             -1.2048854819594377|            -3.0889491836899445|
|    3143|             -0.9883217067643573|              3.889919997813914|
|    3054|             -0.9079612028064519|             4.1149715710151025|
|    3052|             -1.1122950728463596|             -2.556917169064297|
|    3000|                4.65482052280358|              10.16330811857669|
|    3066|              -1.143062691803662|               4.01632192826229|
|    3008|              16.636352030714296|             16.636352030714296|
|    3002|               4.768741414266279|             10.159587087927141|
|    <NA>|             -0.7601763854474433|              4.218058757322852|
|    3184|             -1.0341411680671477|              3.981507453950985|
|    3065|  

In [ ]:
## filter to remove null values from postcode 

In [31]:
print((df1.count()))

154


In [33]:
df = df.withColumnRenamed("School_Count", "Number of Schools")
df = df.withColumnRenamed("groceries_Count", "Number of Groceries")
df = df.withColumnRenamed("hc_Count", "Number of Healthcare Services")
df = df.withColumnRenamed("Postcode_Name", "Suburbs")
df

Postcode,Number of Schools,Number of Groceries,Number of Healthcare Services,Latitude,Longitude,Suburbs
3000,24,42,8,-37.815205,144.96394,Melbourne
3002,14,17,10,-37.816143,144.98045,East Melbourne
3003,7,14,2,-37.81145,144.9254,West Melbourne
3004,7,12,2,-37.83016,144.98045,Melbourne
3006,11,23,6,-37.824547,144.96394,South Wharf
3008,3,3,1,-37.817066,144.94191,Docklands
3010,1,1,NULL,-37.798447,144.9621,Parkville
3011,31,49,14,-37.79602,144.90063,Footscray
3012,9,14,5,-37.814625,144.84563,Tottenham
3013,14,21,3,-37.819813,144.88138,Yarraville


In [35]:

output = df.join(df1,['Postcode'],"inner")
output.show()

+--------+-----------------+-------------------+-----------------------------+----------+---------+----------------+--------------------------------+-------------------------------+
|Postcode|Number of Schools|Number of Groceries|Number of Healthcare Services|  Latitude|Longitude|         Suburbs|1-2 Bedder Affordability Metrics|3+ Bedder Affordability Metrics|
+--------+-----------------+-------------------+-----------------------------+----------+---------+----------------+--------------------------------+-------------------------------+
|    3000|               24|                 42|                            8|-37.815205|144.96394|       Melbourne|                4.65482052280358|              10.16330811857669|
|    3002|               14|                 17|                           10|-37.816143|144.98045|  East Melbourne|               4.768741414266279|             10.159587087927141|
|    3006|               11|                 23|                            6|-37.824547|1